In [2]:
import sys
sys.path.append("/Users/ciaran/Documents/Projects/AFL/git-repositories/afl-player-ratings/")

In [3]:
import pandas as pd
import numpy as np
import joblib
from scipy import stats

from exp_vaep.domain.vaep.vaep import *

import warnings
warnings.filterwarnings('ignore')

pd.options.display.max_rows = 999
pd.options.display.max_columns = 999

%load_ext autoreload
%autoreload 2

Load VAEP Data

In [4]:
exp_vaep_data = pd.read_csv("/Users/ciaran/Documents/Projects/AFL/git-repositories/afl-player-ratings/exp_vaep/data/exp_vaep_values_v2.csv")
exp_vaep_data.head()

,match_id,chain_number,order,quarter,quarter_seconds,overall_seconds,team,player,start_x,start_y,end_x,end_y,action_type,outcome_type,xScore,exp_scores,exp_concedes,exp_offensive_value,exp_defensive_value,exp_vaep_value
0,202101_BrisbaneLions_Sydney,1,2.0,1,24.0,24.0,Brisbane Lions,Dayne Zorko,8.0,-5.0,9.0,-6.0,Hard Ball Get,effective,NaN,0.359902,0.069447,0.000000,-0.000000,0.000000
1,202101_BrisbaneLions_Sydney,1,3.0,1,24.0,24.0,Brisbane Lions,Dayne Zorko,9.0,-6.0,-11.0,7.0,Handball,ineffective,NaN,0.000000,0.094798,-0.359902,-0.025351,-0.385253
2,202101_BrisbaneLions_Sydney,1,4.0,1,28.0,28.0,Sydney,Oliver Florent,-11.0,7.0,-12.0,5.0,Loose Ball Get,effective,NaN,0.271695,0.118642,0.176897,-0.118642,0.058255
3,202101_BrisbaneLions_Sydney,1,5.0,1,29.0,29.0,Sydney,Oliver Florent,-12.0,5.0,-14.0,2.0,Handball,effective,NaN,0.376785,0.103702,0.105090,0.014940,0.120030
4,202101_BrisbaneLions_Sydney,1,6.0,1,30.0,30.0,Sydney,George Hewett,-14.0,2.0,-22.0,2.0,Carry,effective,NaN,0.190404,0.175683,-0.186381,-0.071980,-0.258362


In [5]:
exp_vaep_data[['xScore','exp_scores', 'exp_concedes', 'exp_vaep_value', 'exp_offensive_value', 'exp_defensive_value']].sum()

xScore                 101699.411921
exp_scores             806226.625854
exp_concedes           166496.158793
exp_vaep_value         108758.287080
exp_offensive_value    106964.241339
exp_defensive_value      1794.045789
dtype: float64

In [6]:
exp_vaep_data['Season'] = exp_vaep_data['match_id'].apply(lambda x: int(x[:4]))
exp_vaep_2021 = exp_vaep_data[exp_vaep_data['Season'] == 2021]
exp_vaep_2022 = exp_vaep_data[exp_vaep_data['Season'] == 2022]
exp_vaep_2023 = exp_vaep_data[exp_vaep_data['Season'] == 2023]

In [7]:
player_stats = pd.read_csv("/Users/ciaran/Documents/Projects/AFL/git-repositories/afl-player-ratings/data/player_stats.csv")

In [8]:
player_stats['Season'] = player_stats['Match_ID'].apply(lambda x: int(x[:4]))
player_stats_2021 = player_stats[player_stats['Season'] == 2021]
player_stats_2022 = player_stats[player_stats['Season'] == 2022]
player_stats_2023 = player_stats[player_stats['Season'] == 2023]

Individual Actions

In [9]:
exp_vaep_data['exp_vaep_value'].max(), exp_vaep_data['exp_vaep_value'].min()

(7.4563103, -4.663542)

In [10]:
exp_vaep_data.sort_values('exp_vaep_value', ascending = False).head()

,match_id,chain_number,order,quarter,quarter_seconds,overall_seconds,team,player,start_x,start_y,end_x,end_y,action_type,outcome_type,xScore,exp_scores,exp_concedes,exp_offensive_value,exp_defensive_value,exp_vaep_value,Season
316762,202123_Adelaide_NorthMelbourne,194,1633.0,4,194.0,5540.0,North Melbourne,Nick Larkey,73.0,0.0,69.0,0.0,Free For,effective,NaN,5.279390,0.117992,4.803880,2.652430,7.456310,2021
508992,202211_WestCoast_WesternBulldogs,168,1582.0,4,469.0,6150.0,Western Bulldogs,Alex Keath,62.0,0.0,62.0,0.0,Shot,effective,5.632503,5.327965,0.659337,4.553743,1.002476,5.556219,2022
349807,202201_Carlton_Richmond,202,1775.0,4,1357.0,6696.0,Carlton,Adam Cerra,57.0,7.0,57.0,7.0,Shot,effective,5.357392,5.158351,0.434996,4.530820,0.929322,5.460142,2022
79869,202106_Fremantle_NorthMelbourne,233,2123.0,4,1587.0,6987.0,Fremantle,Matt Taberner,69.0,0.0,64.0,0.0,Free For,effective,NaN,5.525512,0.099236,5.525512,-0.099236,5.426276,2021
623687,202220_Richmond_BrisbaneLions,30,273.0,1,917.0,917.0,Brisbane Lions,Eric Hipwood,75.0,0.0,70.0,0.0,Free For,effective,NaN,5.441927,0.091920,5.441927,-0.091920,5.350007,2022


In [12]:
exp_vaep_data.loc[316760:316765]

,match_id,chain_number,order,quarter,quarter_seconds,overall_seconds,team,player,start_x,start_y,end_x,end_y,action_type,outcome_type,xScore,exp_scores,exp_concedes,exp_offensive_value,exp_defensive_value,exp_vaep_value,Season
316760,202123_Adelaide_NorthMelbourne,193,1629.0,4,134.0,5480.0,Adelaide,Brayden Cook,-4.0,47.0,-4.0,47.0,Kick,clanger,NaN,0.197219,0.105079,-0.222430,-0.059150,-0.281581,2021
316761,202123_Adelaide_NorthMelbourne,193,1632.0,4,136.0,5482.0,North Melbourne,Aaron Hall,-42.0,-25.0,73.0,0.0,Uncontested Mark,effective,NaN,0.475510,2.770423,0.370431,-2.573203,-2.202772,2021
316762,202123_Adelaide_NorthMelbourne,194,1633.0,4,194.0,5540.0,North Melbourne,Nick Larkey,73.0,0.0,69.0,0.0,Free For,effective,NaN,5.279390,0.117992,4.803880,2.652430,7.456310,2021
316763,202123_Adelaide_NorthMelbourne,194,1634.0,4,208.0,5554.0,North Melbourne,Nick Larkey,69.0,0.0,69.0,0.0,Shot,effective,5.731496,5.476623,0.303302,0.197233,-0.185310,0.011923,2021
316764,202123_Adelaide_NorthMelbourne,195,1637.0,4,278.0,5624.0,Adelaide,Ben Keays,0.0,-17.0,-4.0,26.0,Kick,clanger,NaN,0.326958,0.337775,0.326958,-0.337775,-0.010817,2021
316765,202123_Adelaide_NorthMelbourne,195,1638.0,4,279.0,5625.0,North Melbourne,Trent Dumont,-4.0,26.0,-2.0,24.0,Gather,effective,NaN,0.108294,0.077869,-0.229481,0.249089,0.019608,2021


In [10]:
exp_vaep_data.sort_values('exp_vaep_value', ascending = True).head()

,match_id,chain_number,order,quarter,quarter_seconds,overall_seconds,team,player,start_x,start_y,end_x,end_y,action_type,outcome_type,xScore,exp_scores,exp_concedes,exp_offensive_value,exp_defensive_value,exp_vaep_value,Season
19920,202102_GoldCoast_NorthMelbourne,200,1728.0,4,63.0,5614.0,North Melbourne,Jared Polec,-69.0,-1.0,75.0,0.0,Loose Ball Get,effective,NaN,0.101595,5.180285,0.073832,-4.737373,-4.663542,2021
1028628,202324_PortAdelaide_Richmond,222,1875.0,4,1006.0,6693.0,Richmond,Nathan Broad,-75.0,-6.0,76.0,0.0,Spoil,effective,NaN,0.078161,4.649190,0.078161,-4.649190,-4.571029,2023
733521,202304_Adelaide_Fremantle,217,1686.0,4,687.0,6204.0,Fremantle,Alex Pearce,-80.0,-6.0,80.0,1.0,Spoil,effective,NaN,0.135475,4.433515,0.135475,-4.433515,-4.298040,2023
898899,202316_Adelaide_NorthMelbourne,269,2118.0,4,1643.0,7938.0,North Melbourne,Aidan Corr,-78.0,-4.0,78.0,4.0,Spoil,effective,NaN,0.058878,4.802732,0.058878,-4.232055,-4.173177,2023
91332,202107_Adelaide_GreaterWesternSydney,176,1569.0,3,1697.0,5080.0,Greater Western Sydney,Connor Idun,-77.0,-0.0,80.0,-4.0,Spoil,effective,NaN,0.000000,4.167290,0.000000,-4.167290,-4.167290,2021


Match Level

In [11]:
def get_match_player_level_vaep(vaep_data, match_id, shots = True):
    
    match = get_match(vaep_data, match_id)
    if shots == False:
        match = match[match['action_type'] != "Shot"]
    match_vaep = get_player_vaep(match)
    match_vaep_ranking = get_match_rankings(match_vaep)
    
    return match_vaep_ranking

In [12]:
match_id = "2022F4_Geelong_Sydney"

In [13]:
match_vaep_ranking = get_match_player_level_vaep(exp_vaep_data, match_id)
match_vaep_ranking.sort_values('exp_vaep_value', ascending = False).head()

,player,team,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions,vaep_ranking,offensive_ranking,defensive_ranking,normalised_exp_vaep_value,normalised_exp_offensive_value,normalised_exp_defensive_value,z_exp_vaep_value,z_exp_offensive_value,z_exp_defensive_value
30,Patrick Dangerfield,Geelong,15.696886,15.664802,0.032084,58,1.0,1.0,19.0,1.000000,1.000000,0.440498,3.328965,3.582580,0.183025
24,Mark Blicavs,Geelong,15.555391,13.423626,2.131765,49,2.0,2.0,5.0,0.992543,0.865812,0.694816,3.292052,2.938175,1.459669
5,Chad Warner,Sydney,9.248642,10.645956,-1.397314,69,3.0,3.0,36.0,0.660174,0.699503,0.267366,1.646793,2.139511,-0.686075
13,Isaac Smith,Geelong,7.629659,8.877090,-1.247431,69,4.0,4.0,34.0,0.574852,0.593594,0.285520,1.224444,1.630909,-0.594944
45,Zach Tuohy,Geelong,6.944715,2.293307,4.651408,49,5.0,23.0,1.0,0.538755,0.199398,1.000000,1.045761,-0.262125,2.991658


In [14]:
match_noshot_vaep_ranking = get_match_player_level_vaep(exp_vaep_data, match_id, shots=False)
match_noshot_vaep_ranking.sort_values('exp_vaep_value', ascending = False).head()

,player,team,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions,vaep_ranking,offensive_ranking,defensive_ranking,normalised_exp_vaep_value,normalised_exp_offensive_value,normalised_exp_defensive_value,z_exp_vaep_value,z_exp_offensive_value,z_exp_defensive_value
30,Patrick Dangerfield,Geelong,15.252678,15.260346,-0.007668,57,1.0,1.0,20.0,1.000000,1.000000,0.435683,3.402391,3.845663,0.075200
24,Mark Blicavs,Geelong,15.035426,12.693676,2.341750,47,2.0,2.0,5.0,0.988276,0.848092,0.720249,3.343375,3.049904,1.533127
5,Chad Warner,Sydney,7.922794,8.609402,-0.686608,66,3.0,3.0,33.0,0.604451,0.606364,0.353448,1.411263,1.783634,-0.346115
45,Zach Tuohy,Geelong,6.944715,2.293307,4.651408,49,4.0,22.0,1.0,0.551670,0.232546,1.000000,1.145572,-0.174580,2.966381
23,Luke Parker,Sydney,6.791352,5.612276,1.179075,49,5.0,6.0,8.0,0.543394,0.428979,0.579424,1.103912,0.854419,0.811631


In [15]:
match_stats = player_stats[player_stats['Match_ID'] == match_id]
match_stats[['Team', 'Player', "Goals", "Behinds", "Disposals"]]

,Team,Player,Goals,Behinds,Disposals
18967,Geelong,Brad Close,2,0,18
18968,Geelong,Brandan Parfitt,1,0,8
18969,Geelong,Cameron Guthrie,1,1,16
18970,Geelong,Gary Rohan,0,1,7
18971,Geelong,Gryan Miers,0,1,18
18972,Geelong,Isaac Smith,3,1,32
18973,Geelong,Jack Henry,0,0,10
18974,Geelong,Jake Kolodjashnij,0,0,17
18975,Geelong,Jed Bews,0,0,6
18976,Geelong,Jeremy Cameron,2,1,18


Player Ratings

In [16]:
players_vaep = get_player_vaep(exp_vaep_data)
players_vaep.sort_values('exp_vaep_value', ascending = False).head(5)

,player,team,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions
584,Marcus Bontempelli,Western Bulldogs,686.520275,705.721933,-19.201658,4145
149,Christian Petracca,Melbourne,635.135553,651.111731,-15.976179,4317
791,Shai Bolton,Richmond,544.407256,570.169280,-25.762023,2857
514,Lachie Neale,Brisbane Lions,507.729358,492.753667,14.975689,3940
319,Jack Macrae,Western Bulldogs,499.807719,483.317436,16.490285,4573


In [17]:
players_noshot_vaep = get_player_vaep(exp_vaep_data[exp_vaep_data['action_type'] != "Shot"])
players_noshot_vaep.sort_values('exp_vaep_value', ascending = False).head(10)

,player,team,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions
584,Marcus Bontempelli,Western Bulldogs,651.974412,656.606369,-4.631958,3995
149,Christian Petracca,Melbourne,562.096498,562.638430,-0.541934,4100
791,Shai Bolton,Richmond,497.212759,503.949701,-6.736941,2638
319,Jack Macrae,Western Bulldogs,488.028634,467.041435,20.987200,4538
514,Lachie Neale,Brisbane Lions,483.421909,465.165690,18.256217,3879
151,Clayton Oliver,Melbourne,475.362222,462.511202,12.851020,4140
198,Darcy Parish,Essendon,467.726298,481.592693,-13.866397,3681
858,Tom Liberatore,Western Bulldogs,460.174094,431.861398,28.312697,3593
701,Patrick Cripps,Carlton,459.078967,465.466896,-6.387928,3280
919,Zach Merrett,Essendon,439.994073,433.499552,6.494523,4093


Normalise for minutes played

In [18]:
player_minutes = get_player_minutes(player_stats)
players_vaep_mins = players_vaep.merge(player_minutes, how = 'left', on = "player")
player_ratings = convert_vaep_to_rating(players_vaep_mins)
player_ratings.sort_values('vaep_rating', ascending = False).head(10)

,player,team,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
584,Marcus Bontempelli,Western Bulldogs,686.520275,705.721933,-19.201658,4145,5750.0,68.0,11.939483,12.273425,-0.333942
702,Patrick Dangerfield,Geelong,377.717031,391.274317,-13.557286,2392,3342.0,48.0,11.302125,11.707789,-0.405664
198,Darcy Parish,Essendon,479.300674,495.737992,-16.437318,3723,4298.0,53.0,11.151714,11.534155,-0.382441
149,Christian Petracca,Melbourne,635.135553,651.111731,-15.976179,4317,5937.0,68.0,10.697921,10.967016,-0.269095
239,Elliot Yeo,West Coast,173.464762,167.401807,6.062955,1180,1661.0,24.0,10.443393,10.078375,0.365018
791,Shai Bolton,Richmond,544.407256,570.169280,-25.762023,2857,5258.0,62.0,10.353885,10.843843,-0.489959
701,Patrick Cripps,Carlton,477.031872,489.780101,-12.748228,3364,4858.0,59.0,9.819512,10.081929,-0.262417
151,Clayton Oliver,Melbourne,495.165581,486.374608,8.790972,4198,5052.0,58.0,9.801377,9.627368,0.174010
514,Lachie Neale,Brisbane Lions,507.729358,492.753667,14.975689,3940,5185.0,60.0,9.792273,9.503446,0.288827
925,Zak Butters,Port Adelaide,364.843078,367.020999,-2.177920,2858,3966.0,51.0,9.199271,9.254186,-0.054915


In [19]:
player_minutes = get_player_minutes(player_stats)
players_noshot_vaep_mins = players_noshot_vaep.merge(player_minutes, how = 'left', on = "player")
player_noshot_ratings = convert_vaep_to_rating(players_noshot_vaep_mins)
player_noshot_ratings.sort_values('vaep_rating', ascending = False).head(10)

,player,team,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
584,Marcus Bontempelli,Western Bulldogs,651.974412,656.606369,-4.631958,3995,5750.0,68.0,11.338685,11.419241,-0.080556
198,Darcy Parish,Essendon,467.726298,481.592693,-13.866397,3681,4298.0,53.0,10.882417,11.205042,-0.322624
702,Patrick Dangerfield,Geelong,359.113024,367.995940,-8.882916,2319,3342.0,48.0,10.745453,11.011249,-0.265796
239,Elliot Yeo,West Coast,169.074643,162.213868,6.860774,1165,1661.0,24.0,10.179087,9.766037,0.413051
149,Christian Petracca,Melbourne,562.096498,562.638430,-0.541934,4100,5937.0,68.0,9.467686,9.476814,-0.009128
791,Shai Bolton,Richmond,497.212759,503.949701,-6.736941,2638,5258.0,62.0,9.456310,9.584437,-0.128127
701,Patrick Cripps,Carlton,459.078967,465.466896,-6.387928,3280,4858.0,59.0,9.449958,9.581451,-0.131493
151,Clayton Oliver,Melbourne,475.362222,462.511202,12.851020,4140,5052.0,58.0,9.409387,9.155012,0.254375
514,Lachie Neale,Brisbane Lions,483.421909,465.165690,18.256217,3879,5185.0,60.0,9.323470,8.971373,0.352097
437,Joel Selwood,Geelong,287.695544,289.837306,-2.141762,2160,3233.0,44.0,8.898718,8.964965,-0.066247


In [20]:
player_ratings.sort_values('offensive_rating', ascending = False).head(10)

,player,team,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
584,Marcus Bontempelli,Western Bulldogs,686.520275,705.721933,-19.201658,4145,5750.0,68.0,11.939483,12.273425,-0.333942
702,Patrick Dangerfield,Geelong,377.717031,391.274317,-13.557286,2392,3342.0,48.0,11.302125,11.707789,-0.405664
198,Darcy Parish,Essendon,479.300674,495.737992,-16.437318,3723,4298.0,53.0,11.151714,11.534155,-0.382441
149,Christian Petracca,Melbourne,635.135553,651.111731,-15.976179,4317,5937.0,68.0,10.697921,10.967016,-0.269095
791,Shai Bolton,Richmond,544.407256,570.169280,-25.762023,2857,5258.0,62.0,10.353885,10.843843,-0.489959
142,Charlie Curnow,Carlton,375.846295,416.754772,-40.908478,1378,4112.0,45.0,9.140231,10.135087,-0.994856
701,Patrick Cripps,Carlton,477.031872,489.780101,-12.748228,3364,4858.0,59.0,9.819512,10.081929,-0.262417
239,Elliot Yeo,West Coast,173.464762,167.401807,6.062955,1180,1661.0,24.0,10.443393,10.078375,0.365018
856,Tom J. Lynch,Richmond,298.146914,329.180098,-31.033186,966,3282.0,37.0,9.084306,10.029863,-0.945557
817,Taylor Walker,Adelaide,384.024580,434.259180,-50.234601,1592,4408.0,52.0,8.711991,9.851615,-1.139623


In [21]:
player_noshot_ratings.sort_values('offensive_rating', ascending = False).head(10)

,player,team,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
584,Marcus Bontempelli,Western Bulldogs,651.974412,656.606369,-4.631958,3995,5750.0,68.0,11.338685,11.419241,-0.080556
198,Darcy Parish,Essendon,467.726298,481.592693,-13.866397,3681,4298.0,53.0,10.882417,11.205042,-0.322624
702,Patrick Dangerfield,Geelong,359.113024,367.995940,-8.882916,2319,3342.0,48.0,10.745453,11.011249,-0.265796
239,Elliot Yeo,West Coast,169.074643,162.213868,6.860774,1165,1661.0,24.0,10.179087,9.766037,0.413051
791,Shai Bolton,Richmond,497.212759,503.949701,-6.736941,2638,5258.0,62.0,9.456310,9.584437,-0.128127
701,Patrick Cripps,Carlton,459.078967,465.466896,-6.387928,3280,4858.0,59.0,9.449958,9.581451,-0.131493
149,Christian Petracca,Melbourne,562.096498,562.638430,-0.541934,4100,5937.0,68.0,9.467686,9.476814,-0.009128
151,Clayton Oliver,Melbourne,475.362222,462.511202,12.851020,4140,5052.0,58.0,9.409387,9.155012,0.254375
856,Tom J. Lynch,Richmond,280.967662,295.120702,-14.153041,792,3282.0,37.0,8.560867,8.992099,-0.431232
514,Lachie Neale,Brisbane Lions,483.421909,465.165690,18.256217,3879,5185.0,60.0,9.323470,8.971373,0.352097


In [22]:
player_ratings.sort_values('defensive_rating', ascending = False).head(10)

,player,team,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
779,Sam Taylor,Greater Western Sydney,308.748004,96.179018,212.568986,2486,4916.0,52.0,6.280472,1.956449,4.324023
845,Tom Clurey,Port Adelaide,117.943792,33.213570,84.730222,810,2210.0,25.0,5.336823,1.502876,3.833947
264,Harris Andrews,Brisbane Lions,425.505536,185.291667,240.213868,3088,6317.0,65.0,6.735880,2.933223,3.802657
757,Sam Collins,Gold Coast,323.710241,124.126894,199.583347,2416,5797.0,61.0,5.584099,2.141226,3.442873
841,Tom Barrass,West Coast,240.993942,89.429858,151.564083,2084,4501.0,49.0,5.354231,1.986889,3.367342
304,Jack Buckley,Greater Western Sydney,187.095303,94.642263,92.453041,1402,2906.0,33.0,6.438242,3.256788,3.181454
480,Josh Worrell,Adelaide,70.871230,39.046858,31.824371,644,1037.0,13.0,6.834256,3.765367,3.068888
349,Jacob Weitering,Carlton,325.304915,152.881196,172.423721,2701,5671.0,59.0,5.736288,2.695842,3.040446
197,Darcy Moore,Collingwood,339.233304,184.618030,154.615274,2646,5218.0,55.0,6.501213,3.538099,2.963114
535,Lewis Melican,Sydney,49.879947,16.767278,33.112670,433,1122.0,13.0,4.445628,1.494410,2.951218


In [23]:
player_noshot_ratings.sort_values('defensive_rating', ascending = False).head(10)

,player,team,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
779,Sam Taylor,Greater Western Sydney,308.748004,96.179018,212.568986,2486,4916.0,52.0,6.280472,1.956449,4.324023
845,Tom Clurey,Port Adelaide,117.943792,33.213570,84.730222,810,2210.0,25.0,5.336823,1.502876,3.833947
264,Harris Andrews,Brisbane Lions,423.528669,182.861798,240.666870,3084,6317.0,65.0,6.704586,2.894757,3.809829
757,Sam Collins,Gold Coast,323.710241,124.126894,199.583347,2416,5797.0,61.0,5.584099,2.141226,3.442873
841,Tom Barrass,West Coast,240.699085,88.956377,151.742707,2083,4501.0,49.0,5.347680,1.976369,3.371311
304,Jack Buckley,Greater Western Sydney,185.292495,93.033451,92.259044,1399,2906.0,33.0,6.376204,3.201426,3.174778
480,Josh Worrell,Adelaide,70.871230,39.046858,31.824371,644,1037.0,13.0,6.834256,3.765367,3.068888
349,Jacob Weitering,Carlton,321.545788,149.866734,171.679055,2699,5671.0,59.0,5.670002,2.642686,3.027315
197,Darcy Moore,Collingwood,339.778521,184.002986,155.775534,2637,5218.0,55.0,6.511662,3.526312,2.985349
535,Lewis Melican,Sydney,49.879947,16.767278,33.112670,433,1122.0,13.0,4.445628,1.494410,2.951218


VAEP by Action

In [24]:
action_vaep = get_vaep_action_summary(exp_vaep_data)
action_vaep.sort_values("exp_vaep_value_per_action")

,action_type,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions,action_%,exp_vaep_value_per_action,off_value_per_action,def_value_per_action,vaep_%,offensive_%,defensive_%
2,Error,-2370.992540,-1336.745172,-1034.247370,3238,0.003137,-0.732240,-0.412831,-0.319409,-0.021801,-0.012497,-0.576489
12,Tackle,-55.013299,-9.230596,-45.782704,314,0.000304,-0.175202,-0.029397,-0.145805,-0.000506,-0.000086,-0.025519
5,Handball,-7799.254371,-2157.869823,-5641.384548,181764,0.176102,-0.042909,-0.011872,-0.031037,-0.071712,-0.020174,-3.144504
7,Kick,9115.646555,17575.906702,-8460.260132,239630,0.232165,0.038041,0.073346,-0.035306,0.083816,0.164316,-4.715744
0,Carry,11212.723776,14524.195646,-3311.471867,163299,0.158212,0.068664,0.088942,-0.020279,0.103098,0.135786,-1.845812
11,Spoil,4213.633523,285.372097,3928.261424,41219,0.039935,0.102226,0.006923,0.095302,0.038743,0.002668,2.189610
4,Gather,5434.267341,5050.784693,383.482644,40992,0.039715,0.132569,0.123214,0.009355,0.049966,0.047219,0.213753
6,Hard Ball Get,8365.312980,5315.866623,3049.446353,52402,0.050770,0.159637,0.101444,0.058193,0.076917,0.049698,1.699759
13,Uncontested Mark,20826.886657,19819.813037,1007.073635,114870,0.111292,0.181308,0.172541,0.008767,0.191497,0.185294,0.561342
10,Shot,6028.811551,8784.026607,-2755.215046,30145,0.029206,0.199994,0.291392,-0.091399,0.055433,0.082121,-1.535755


In [25]:
match_data = get_match(exp_vaep_data, match_id)
match_action_vaep = get_vaep_action_summary(match_data)
match_action_vaep.sort_values("exp_vaep_value_per_action")

,action_type,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions,action_%,exp_vaep_value_per_action,off_value_per_action,def_value_per_action,vaep_%,offensive_%,defensive_%
2,Error,-0.988351,-1.167487,0.179136,3,0.001858,-0.329450,-0.389162,0.059712,-0.007318,-0.007919,-0.014480
5,Handball,-7.304367,-1.524661,-5.779706,280,0.173375,-0.026087,-0.005445,-0.020642,-0.054084,-0.010342,0.467199
11,Spoil,-0.667707,1.011051,-1.678758,58,0.035913,-0.011512,0.017432,-0.028944,-0.004944,0.006858,0.135701
0,Carry,2.322082,10.115521,-7.793438,265,0.164087,0.008763,0.038172,-0.029409,0.017193,0.068613,0.629978
7,Kick,24.428041,33.570056,-9.142016,374,0.231579,0.065316,0.089760,-0.024444,0.180872,0.227705,0.738989
4,Gather,4.798633,5.314939,-0.516306,59,0.036533,0.081333,0.090084,-0.008751,0.035530,0.036051,0.041735
6,Hard Ball Get,8.239417,6.708850,1.530568,80,0.049536,0.102993,0.083861,0.019132,0.061007,0.045506,-0.123722
12,Uncontested Mark,25.814932,24.932737,0.882195,193,0.119505,0.133756,0.129185,0.004571,0.191141,0.169118,-0.071312
9,Loose Ball Get,30.783573,20.878195,9.905378,190,0.117647,0.162019,0.109885,0.052134,0.227931,0.141616,-0.800695
10,Shot,9.589396,16.033228,-6.443832,47,0.029102,0.204030,0.341133,-0.137103,0.071003,0.108753,0.520883


In [26]:
player = "Christian Petracca"
player_data = get_player(exp_vaep_data, player)
player_action_vaep = get_vaep_action_summary(player_data)
player_action_vaep.sort_values("exp_vaep_value")

,action_type,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions,action_%,exp_vaep_value_per_action,off_value_per_action,def_value_per_action,vaep_%,offensive_%,defensive_%
5,Handball,-34.777072,-17.213722,-17.563350,959,0.222145,-0.036264,-0.017950,-0.018314,-0.054755,-0.026437,1.099346
2,Error,-17.774363,-11.088257,-6.686106,26,0.006023,-0.683629,-0.426471,-0.257158,-0.027985,-0.017030,0.418505
11,Spoil,2.136720,2.146807,-0.010087,18,0.004170,0.118707,0.119267,-0.000560,0.003364,0.003297,0.000631
8,Knock On,3.890435,2.893986,0.996449,12,0.002780,0.324203,0.241166,0.083037,0.006125,0.004445,-0.062371
1,Contested Mark,11.554891,11.064168,0.490723,19,0.004401,0.608152,0.582325,0.025828,0.018193,0.016993,-0.030716
3,Free For,41.820351,37.483483,4.336869,59,0.013667,0.708820,0.635313,0.073506,0.065845,0.057568,-0.271458
7,Kick,52.051817,82.017327,-29.965510,870,0.201529,0.059830,0.094273,-0.034443,0.081954,0.125965,1.875637
6,Hard Ball Get,52.189280,41.929918,10.259362,319,0.073894,0.163603,0.131442,0.032161,0.082170,0.064397,-0.642166
10,Shot,73.039055,88.473301,-15.434245,217,0.050266,0.336586,0.407711,-0.071126,0.114998,0.135880,0.966079
12,Uncontested Mark,83.590989,82.398241,1.192748,332,0.076905,0.251780,0.248187,0.003593,0.131611,0.126550,-0.074658


In [27]:
player = "Nick Daicos"
player_data = get_player(exp_vaep_data, player)
player_action_vaep = get_vaep_action_summary(player_data)
player_action_vaep.sort_values("exp_vaep_value")

,action_type,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions,action_%,exp_vaep_value_per_action,off_value_per_action,def_value_per_action,vaep_%,offensive_%,defensive_%
4,Handball,-39.893144,-10.756984,-29.136160,583,0.212154,-0.068427,-0.018451,-0.049976,-0.189236,-0.044447,0.933616
1,Error,-6.173316,-3.022032,-3.151284,9,0.003275,-0.685924,-0.335781,-0.350143,-0.029284,-0.012487,0.100977
10,Spoil,-0.755325,-0.010757,-0.744569,8,0.002911,-0.094416,-0.001345,-0.093071,-0.003583,-0.000044,0.023858
7,Knock On,0.957154,0.732742,0.224411,6,0.002183,0.159526,0.122124,0.037402,0.004540,0.003028,-0.007191
5,Hard Ball Get,14.052289,11.156232,2.896056,77,0.028020,0.182497,0.144886,0.037611,0.066658,0.046096,-0.092799
9,Shot,15.079517,19.845376,-4.765859,56,0.020378,0.269277,0.354382,-0.085105,0.071531,0.081999,0.152713
3,Gather,22.654583,17.131477,5.523107,103,0.037482,0.219947,0.166325,0.053622,0.107464,0.070786,-0.176978
2,Free For,24.943760,20.081064,4.862696,52,0.018923,0.479688,0.386174,0.093513,0.118323,0.082973,-0.155816
6,Kick,30.438155,48.451919,-18.013764,630,0.229258,0.048315,0.076908,-0.028593,0.144386,0.200198,0.577219
11,Uncontested Mark,31.345257,30.715450,0.629808,194,0.070597,0.161573,0.158327,0.003246,0.148689,0.126913,-0.020181


In [28]:
player = "Darcy Moore"
player_data = get_player(exp_vaep_data, player)
player_action_vaep = get_vaep_action_summary(player_data)
player_action_vaep.sort_values("exp_vaep_value")

,action_type,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions,action_%,exp_vaep_value_per_action,off_value_per_action,def_value_per_action,vaep_%,offensive_%,defensive_%
2,Error,-8.505646,-0.484508,-8.021139,6,0.002268,-1.417608,-0.080751,-1.336856,-0.025073,-0.002624,-0.051878
5,Handball,-5.399995,0.271107,-5.671102,294,0.111111,-0.018367,0.000922,-0.019289,-0.015918,0.001468,-0.036679
0,Carry,-3.733174,27.408363,-31.141537,309,0.116780,-0.012081,0.088700,-0.100782,-0.011005,0.148460,-0.201413
10,Shot,-0.545217,0.615044,-1.160261,9,0.003401,-0.060580,0.068338,-0.128918,-0.001607,0.003331,-0.007504
4,Gather,1.650296,1.297342,0.352954,48,0.018141,0.034381,0.027028,0.007353,0.004865,0.007027,0.002283
7,Kick,10.788421,50.927287,-40.138866,616,0.232804,0.017514,0.082674,-0.065160,0.031802,0.275852,-0.259605
8,Knock On,11.448492,4.172752,7.275741,59,0.022298,0.194042,0.070725,0.123318,0.033748,0.022602,0.047057
6,Hard Ball Get,19.016170,3.734196,15.281974,104,0.039305,0.182848,0.035906,0.146942,0.056056,0.020227,0.098839
3,Free For,29.288985,12.988155,16.300830,42,0.015873,0.697357,0.309242,0.388115,0.086339,0.070351,0.105428
9,Loose Ball Get,35.199146,7.799531,27.399616,209,0.078987,0.168417,0.037318,0.131099,0.103761,0.042247,0.177212


In [29]:
player = "Taylor Walker"
player_data = get_player(exp_vaep_data, player)
player_action_vaep = get_vaep_action_summary(player_data)
player_action_vaep.sort_values("exp_vaep_value").round(2)

,action_type,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions,action_%,exp_vaep_value_per_action,off_value_per_action,def_value_per_action,vaep_%,offensive_%,defensive_%
5,Handball,-35.55,-34.56,-0.99,212,0.13,-0.17,-0.16,-0.00,-0.09,-0.08,0.02
11,Spoil,-9.25,-7.18,-2.07,36,0.02,-0.26,-0.20,-0.06,-0.02,-0.02,0.04
2,Error,-6.05,-4.41,-1.63,7,0.00,-0.86,-0.63,-0.23,-0.02,-0.01,0.03
0,Carry,6.86,8.35,-1.48,124,0.08,0.06,0.07,-0.01,0.02,0.02,0.03
4,Gather,7.17,7.34,-0.17,45,0.03,0.16,0.16,-0.00,0.02,0.02,0.00
8,Knock On,9.25,9.04,0.20,20,0.01,0.46,0.45,0.01,0.02,0.02,-0.00
6,Hard Ball Get,21.63,21.73,-0.10,114,0.07,0.19,0.19,-0.00,0.06,0.05,0.00
7,Kick,33.65,38.64,-5.00,243,0.15,0.14,0.16,-0.02,0.09,0.09,0.10
10,Shot,46.93,82.46,-35.53,295,0.19,0.16,0.28,-0.12,0.12,0.19,0.71
9,Loose Ball Get,51.18,51.59,-0.42,127,0.08,0.40,0.41,-0.00,0.13,0.12,0.01


By Season

In [30]:
players_2021 = get_player_vaep(exp_vaep_2021)
players_2022 = get_player_vaep(exp_vaep_2022)
players_2023 = get_player_vaep(exp_vaep_2023)

In [31]:
player_minutes_2021 = get_player_minutes(player_stats_2021)
player_minutes_2022 = get_player_minutes(player_stats_2022)
player_minutes_2023 = get_player_minutes(player_stats_2023)

In [32]:
players_2021 = players_2021.merge(player_minutes_2021, how = "left", on = "player")
players_2022 = players_2022.merge(player_minutes_2022, how = "left", on = "player")
players_2023 = players_2023.merge(player_minutes_2023, how = "left", on = "player")

In [33]:
player_ratings_2021 = convert_vaep_to_rating(players_2021)
player_ratings_2022 = convert_vaep_to_rating(players_2022)
player_ratings_2023 = convert_vaep_to_rating(players_2023)

2021

In [34]:
player_ratings_2021.sort_values('exp_vaep_value', ascending = False).head(10)

,player,team,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
232,Jack Macrae,Western Bulldogs,233.594187,232.345635,1.248553,1872,2154,26,10.844670,10.786705,0.057964
420,Marcus Bontempelli,Western Bulldogs,225.889831,237.890196,-12.000365,1569,2193,26,10.300494,10.847706,-0.547212
110,Christian Petracca,Melbourne,216.011730,233.086045,-17.074316,1524,2237,25,9.656313,10.419582,-0.763268
112,Clayton Oliver,Melbourne,209.875776,213.564361,-3.688585,1702,2211,25,9.492346,9.659175,-0.166829
145,Darcy Parish,Essendon,209.566968,211.426402,-1.859434,1492,1890,23,11.088199,11.186582,-0.098383
154,Dayne Zorko,Brisbane Lions,199.322485,200.110820,-0.788333,1199,1900,23,10.490657,10.532148,-0.041491
618,Tom Liberatore,Western Bulldogs,175.518756,172.267080,3.251675,1284,1967,25,8.923170,8.757859,0.165311
492,Ollie Wines,Port Adelaide,174.879993,185.896302,-11.016308,1648,1855,24,9.427493,10.021364,-0.593871
316,Joel Selwood,Geelong,173.255757,174.725418,-1.469662,1178,1724,23,10.049638,10.134885,-0.085247
100,Charlie Cameron,Brisbane Lions,168.619045,181.693336,-13.074291,560,2082,24,8.098897,8.726865,-0.627968


In [35]:
player_ratings_2021.sort_values('vaep_rating', ascending = False).head(10)

,player,team,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
503,Patrick Dangerfield,Geelong,120.117049,123.127244,-3.010194,749,1080,15,11.121949,11.400671,-0.278722
145,Darcy Parish,Essendon,209.566968,211.426402,-1.859434,1492,1890,23,11.088199,11.186582,-0.098383
268,Jake Stringer,Essendon,168.588379,179.932981,-11.344602,661,1526,19,11.047731,11.791152,-0.743421
232,Jack Macrae,Western Bulldogs,233.594187,232.345635,1.248553,1872,2154,26,10.844670,10.786705,0.057964
154,Dayne Zorko,Brisbane Lions,199.322485,200.110820,-0.788333,1199,1900,23,10.490657,10.532148,-0.041491
420,Marcus Bontempelli,Western Bulldogs,225.889831,237.890196,-12.000365,1569,2193,26,10.300494,10.847706,-0.547212
462,Nat Fyfe,Fremantle,123.659307,117.448750,6.210556,790,1220,15,10.136009,9.626947,0.509062
316,Joel Selwood,Geelong,173.255757,174.725418,-1.469662,1178,1724,23,10.049638,10.134885,-0.085247
42,Ben Cunnington,North Melbourne,123.516348,118.634934,4.881415,880,1237,15,9.985153,9.590536,0.394617
97,Chad Wingard,Hawthorn,120.553727,127.554851,-7.001123,756,1239,16,9.729921,10.294984,-0.565062


In [36]:
player_ratings_2021.sort_values('exp_offensive_value', ascending = False).head(10)

,player,team,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
420,Marcus Bontempelli,Western Bulldogs,225.889831,237.890196,-12.000365,1569,2193,26,10.300494,10.847706,-0.547212
110,Christian Petracca,Melbourne,216.011730,233.086045,-17.074316,1524,2237,25,9.656313,10.419582,-0.763268
232,Jack Macrae,Western Bulldogs,233.594187,232.345635,1.248553,1872,2154,26,10.844670,10.786705,0.057964
112,Clayton Oliver,Melbourne,209.875776,213.564361,-3.688585,1702,2211,25,9.492346,9.659175,-0.166829
145,Darcy Parish,Essendon,209.566968,211.426402,-1.859434,1492,1890,23,11.088199,11.186582,-0.098383
154,Dayne Zorko,Brisbane Lions,199.322485,200.110820,-0.788333,1199,1900,23,10.490657,10.532148,-0.041491
492,Ollie Wines,Port Adelaide,174.879993,185.896302,-11.016308,1648,1855,24,9.427493,10.021364,-0.593871
100,Charlie Cameron,Brisbane Lions,168.619045,181.693336,-13.074291,560,2082,24,8.098897,8.726865,-0.627968
268,Jake Stringer,Essendon,168.588379,179.932981,-11.344602,661,1526,19,11.047731,11.791152,-0.743421
312,Joe Daniher,Brisbane Lions,167.076194,177.846156,-10.769963,742,2252,24,7.419014,7.897254,-0.478240


In [37]:
player_ratings_2021.sort_values('offensive_rating', ascending = False).head(10)

,player,team,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
268,Jake Stringer,Essendon,168.588379,179.932981,-11.344602,661,1526,19,11.047731,11.791152,-0.743421
503,Patrick Dangerfield,Geelong,120.117049,123.127244,-3.010194,749,1080,15,11.121949,11.400671,-0.278722
145,Darcy Parish,Essendon,209.566968,211.426402,-1.859434,1492,1890,23,11.088199,11.186582,-0.098383
420,Marcus Bontempelli,Western Bulldogs,225.889831,237.890196,-12.000365,1569,2193,26,10.300494,10.847706,-0.547212
232,Jack Macrae,Western Bulldogs,233.594187,232.345635,1.248553,1872,2154,26,10.844670,10.786705,0.057964
154,Dayne Zorko,Brisbane Lions,199.322485,200.110820,-0.788333,1199,1900,23,10.490657,10.532148,-0.041491
110,Christian Petracca,Melbourne,216.011730,233.086045,-17.074316,1524,2237,25,9.656313,10.419582,-0.763268
97,Chad Wingard,Hawthorn,120.553727,127.554851,-7.001123,756,1239,16,9.729921,10.294984,-0.565062
316,Joel Selwood,Geelong,173.255757,174.725418,-1.469662,1178,1724,23,10.049638,10.134885,-0.085247
492,Ollie Wines,Port Adelaide,174.879993,185.896302,-11.016308,1648,1855,24,9.427493,10.021364,-0.593871


In [38]:
player_ratings_2021.sort_values('exp_defensive_value', ascending = False).head(10)

,player,team,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
194,Harris Andrews,Brisbane Lions,167.726670,64.299557,103.427113,1109,2224,23,7.541667,2.891167,4.650500
557,Sam Taylor,Greater Western Sydney,105.916926,33.370002,72.546923,829,1781,19,5.947048,1.873667,4.073381
391,Liam Jones,Carlton,125.666045,54.092490,71.573555,716,1814,19,6.927566,2.981945,3.945620
18,Aliir Aliir,Port Adelaide,144.190196,73.250097,70.940100,1042,2315,24,6.228518,3.164151,3.064367
13,Alex Keath,Western Bulldogs,111.851548,42.889002,68.962547,803,2173,23,5.147333,1.973723,3.173610
617,Tom Jonas,Port Adelaide,117.136092,49.469676,67.666416,985,2297,24,5.099525,2.153665,2.945861
253,Jacob Weitering,Carlton,134.250390,67.457147,66.793244,1063,2110,22,6.362578,3.197021,3.165557
540,Sam Collins,Gold Coast,87.961802,28.972037,58.989766,725,1788,20,4.919564,1.620360,3.299204
263,Jake Lever,Melbourne,137.026684,78.497303,58.529381,1141,2472,25,5.543151,3.175457,2.367693
99,Charlie Ballard,Gold Coast,87.107024,34.697486,52.409537,802,1947,21,4.473910,1.782100,2.691810


In [39]:
player_ratings_2021.sort_values('defensive_rating', ascending = False).head(10)

,player,team,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
194,Harris Andrews,Brisbane Lions,167.726670,64.299557,103.427113,1109,2224,23,7.541667,2.891167,4.650500
557,Sam Taylor,Greater Western Sydney,105.916926,33.370002,72.546923,829,1781,19,5.947048,1.873667,4.073381
391,Liam Jones,Carlton,125.666045,54.092490,71.573555,716,1814,19,6.927566,2.981945,3.945620
540,Sam Collins,Gold Coast,87.961802,28.972037,58.989766,725,1788,20,4.919564,1.620360,3.299204
13,Alex Keath,Western Bulldogs,111.851548,42.889002,68.962547,803,2173,23,5.147333,1.973723,3.173610
253,Jacob Weitering,Carlton,134.250390,67.457147,66.793244,1063,2110,22,6.362578,3.197021,3.165557
18,Aliir Aliir,Port Adelaide,144.190196,73.250097,70.940100,1042,2315,24,6.228518,3.164151,3.064367
196,Harrison Petty,Melbourne,74.273755,23.932361,50.341392,511,1682,19,4.415800,1.422851,2.992948
617,Tom Jonas,Port Adelaide,117.136092,49.469676,67.666416,985,2297,24,5.099525,2.153665,2.945861
487,Noah Balta,Richmond,78.466958,47.828179,30.638780,480,1118,14,7.018511,4.278012,2.740499


2022

In [40]:
player_ratings_2022.sort_values('exp_vaep_value', ascending = False).head(10)

,player,team,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
110,Christian Petracca,Melbourne,214.952376,206.353435,8.598939,1416,2108,24,10.196982,9.789062,0.407919
432,Marcus Bontempelli,Western Bulldogs,203.366382,210.471318,-7.104935,1158,1863,22,10.916070,11.297441,-0.381371
383,Lachie Neale,Brisbane Lions,203.223559,201.781136,1.442422,1621,2182,25,9.313637,9.247531,0.066105
582,Shai Bolton,Richmond,196.102443,216.016086,-19.913642,903,1985,23,9.879216,10.882422,-1.003206
112,Clayton Oliver,Melbourne,190.220977,179.778248,10.442730,1629,1995,23,9.534886,9.011441,0.523445
514,Patrick Cripps,Carlton,189.216778,196.911826,-7.695046,1252,1707,21,11.084756,11.535549,-0.450794
626,Tom J. Lynch,Richmond,186.948116,204.977454,-18.029339,514,1601,19,11.676959,12.803089,-1.126130
97,Chad Warner,Sydney,184.897242,195.879415,-10.982173,1208,1938,24,9.540621,10.107297,-0.566676
638,Touk Miller,Gold Coast,177.088024,198.602776,-21.514752,1320,1907,22,9.286210,10.414409,-1.128199
211,Isaac Heeney,Sydney,168.198400,179.418116,-11.219716,896,2136,25,7.874457,8.399725,-0.525268


In [41]:
player_ratings_2022.sort_values('vaep_rating', ascending = False).head(10)

,player,team,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
626,Tom J. Lynch,Richmond,186.948116,204.977454,-18.029339,514,1601,19,11.676959,12.803089,-1.126130
514,Patrick Cripps,Carlton,189.216778,196.911826,-7.695046,1252,1707,21,11.084756,11.535549,-0.450794
432,Marcus Bontempelli,Western Bulldogs,203.366382,210.471318,-7.104935,1158,1863,22,10.916070,11.297441,-0.381371
145,Darcy Parish,Essendon,135.060652,137.815194,-2.754541,1061,1299,16,10.397279,10.609330,-0.212051
157,Dion Prestia,Richmond,137.612929,138.581062,-0.968134,1006,1338,19,10.284972,10.357329,-0.072357
110,Christian Petracca,Melbourne,214.952376,206.353435,8.598939,1416,2108,24,10.196982,9.789062,0.407919
515,Patrick Dangerfield,Geelong,134.733526,133.392270,1.341256,862,1344,19,10.024816,9.925020,0.099796
582,Shai Bolton,Richmond,196.102443,216.016086,-19.913642,903,1985,23,9.879216,10.882422,-1.003206
415,Luke Davies-Uniacke,North Melbourne,163.780113,162.129709,1.650404,1140,1715,21,9.549861,9.453627,0.096233
97,Chad Warner,Sydney,184.897242,195.879415,-10.982173,1208,1938,24,9.540621,10.107297,-0.566676


In [42]:
player_ratings_2022.sort_values('exp_offensive_value', ascending = False).head(10)

,player,team,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
582,Shai Bolton,Richmond,196.102443,216.016086,-19.913642,903,1985,23,9.879216,10.882422,-1.003206
432,Marcus Bontempelli,Western Bulldogs,203.366382,210.471318,-7.104935,1158,1863,22,10.916070,11.297441,-0.381371
110,Christian Petracca,Melbourne,214.952376,206.353435,8.598939,1416,2108,24,10.196982,9.789062,0.407919
626,Tom J. Lynch,Richmond,186.948116,204.977454,-18.029339,514,1601,19,11.676959,12.803089,-1.126130
383,Lachie Neale,Brisbane Lions,203.223559,201.781136,1.442422,1621,2182,25,9.313637,9.247531,0.066105
638,Touk Miller,Gold Coast,177.088024,198.602776,-21.514752,1320,1907,22,9.286210,10.414409,-1.128199
514,Patrick Cripps,Carlton,189.216778,196.911826,-7.695046,1252,1707,21,11.084756,11.535549,-0.450794
97,Chad Warner,Sydney,184.897242,195.879415,-10.982173,1208,1938,24,9.540621,10.107297,-0.566676
104,Charlie Curnow,Carlton,165.115739,183.333298,-18.217560,574,2022,22,8.165961,9.066929,-0.900967
112,Clayton Oliver,Melbourne,190.220977,179.778248,10.442730,1629,1995,23,9.534886,9.011441,0.523445


In [43]:
player_ratings_2022.sort_values('offensive_rating', ascending = False).head(10)

,player,team,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
626,Tom J. Lynch,Richmond,186.948116,204.977454,-18.029339,514,1601,19,11.676959,12.803089,-1.126130
514,Patrick Cripps,Carlton,189.216778,196.911826,-7.695046,1252,1707,21,11.084756,11.535549,-0.450794
432,Marcus Bontempelli,Western Bulldogs,203.366382,210.471318,-7.104935,1158,1863,22,10.916070,11.297441,-0.381371
582,Shai Bolton,Richmond,196.102443,216.016086,-19.913642,903,1985,23,9.879216,10.882422,-1.003206
145,Darcy Parish,Essendon,135.060652,137.815194,-2.754541,1061,1299,16,10.397279,10.609330,-0.212051
638,Touk Miller,Gold Coast,177.088024,198.602776,-21.514752,1320,1907,22,9.286210,10.414409,-1.128199
157,Dion Prestia,Richmond,137.612929,138.581062,-0.968134,1006,1338,19,10.284972,10.357329,-0.072357
97,Chad Warner,Sydney,184.897242,195.879415,-10.982173,1208,1938,24,9.540621,10.107297,-0.566676
164,Dylan Shiel,Essendon,132.364798,141.187280,-8.822482,923,1404,19,9.427692,10.056074,-0.628382
515,Patrick Dangerfield,Geelong,134.733526,133.392270,1.341256,862,1344,19,10.024816,9.925020,0.099796


In [44]:
player_ratings_2022.sort_values('exp_defensive_value', ascending = False).head(10)

,player,team,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
144,Darcy Moore,Collingwood,146.000495,65.437649,80.562845,1044,2314,24,6.309442,2.827902,3.481540
573,Sam Taylor,Greater Western Sydney,113.370306,38.581067,74.789239,1018,2132,22,5.317557,1.809619,3.507938
555,Sam Collins,Gold Coast,114.580290,46.165731,68.414558,798,2154,22,5.319419,2.143256,3.176163
615,Tom Barrass,West Coast,102.438937,34.694543,67.744394,865,1767,19,5.797337,1.963472,3.833865
188,Harris Andrews,Brisbane Lions,106.575717,47.020877,59.554840,948,2337,24,4.560364,2.012019,2.548346
255,Jacob Weitering,Carlton,91.050542,37.324721,53.725822,644,1680,18,5.419675,2.221710,3.197966
558,Sam De Koning,Geelong,90.001511,37.194662,52.806849,825,2114,23,4.257404,1.759445,2.497959
399,Lewis Young,Carlton,75.886734,24.512372,51.374362,824,1872,19,4.053779,1.309422,2.744357
73,Brennan Cox,Fremantle,109.333289,60.503981,48.829310,1024,2000,23,5.466664,3.025199,2.441465
540,Robbie Tarrant,Richmond,76.675544,31.626763,45.048780,615,1829,20,4.192211,1.729183,2.463028


In [45]:
player_ratings_2022.sort_values('defensive_rating', ascending = False).head(10)

,player,team,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
615,Tom Barrass,West Coast,102.438937,34.694543,67.744394,865,1767,19,5.797337,1.963472,3.833865
617,Tom Clurey,Port Adelaide,60.077463,16.036991,44.040472,392,1182,13,5.082696,1.356767,3.725928
573,Sam Taylor,Greater Western Sydney,113.370306,38.581067,74.789239,1018,2132,22,5.317557,1.809619,3.507938
144,Darcy Moore,Collingwood,146.000495,65.437649,80.562845,1044,2314,24,6.309442,2.827902,3.481540
255,Jacob Weitering,Carlton,91.050542,37.324721,53.725822,644,1680,18,5.419675,2.221710,3.197966
555,Sam Collins,Gold Coast,114.580290,46.165731,68.414558,798,2154,22,5.319419,2.143256,3.176163
162,Dylan Grimes,Richmond,58.210083,20.049336,38.160747,481,1228,15,4.740235,1.632682,3.107553
399,Lewis Young,Carlton,75.886734,24.512372,51.374362,824,1872,19,4.053779,1.309422,2.744357
353,Josh Walker,North Melbourne,72.993789,36.662989,36.330800,496,1371,16,5.324128,2.674179,2.649949
188,Harris Andrews,Brisbane Lions,106.575717,47.020877,59.554840,948,2337,24,4.560364,2.012019,2.548346


2023

In [66]:
player_ratings_2023.sort_values('exp_vaep_value', ascending = False).head(10)

,player,team,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
424,Marcus Bontempelli,Western Bulldogs,257.264062,257.360419,-0.096358,1418,1694.0,20.0,15.186780,15.192469,-0.005688
490,Noah Anderson,Gold Coast,207.136345,205.099730,2.036617,1352,1607.0,19.0,12.889629,12.762895,0.126734
111,Christian Petracca,Melbourne,204.171447,211.672250,-7.500802,1377,1592.0,19.0,12.824840,13.295996,-0.471156
573,Shai Bolton,Richmond,197.169424,198.858498,-1.689074,1107,1645.0,19.0,11.985983,12.088662,-0.102679
84,Caleb Serong,Fremantle,192.770327,197.288034,-4.517708,1436,1500.0,18.0,12.851355,13.152536,-0.301181
105,Charlie Curnow,Carlton,192.572502,211.673820,-19.101318,699,1774.0,19.0,10.855271,11.932008,-1.076737
660,Zak Butters,Port Adelaide,187.002842,188.319077,-1.316234,1410,1495.0,19.0,12.508551,12.596594,-0.088042
376,Lachie Neale,Brisbane Lions,170.908705,162.303695,8.605008,1350,1563.0,18.0,10.934658,10.384114,0.550544
578,Stephen Coniglio,Greater Western Sydney,170.722975,186.689445,-15.966470,1354,1481.0,18.0,11.527547,12.605634,-1.078087
657,Zach Merrett,Essendon,168.855597,169.499430,-0.643831,1364,1511.0,18.0,11.175089,11.217699,-0.042610


In [56]:
player_ratings_2023.sort_values('vaep_rating', ascending = False).head(10)

,player,team,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
424,Marcus Bontempelli,Western Bulldogs,257.264062,257.360419,-0.096358,1418,1694.0,20.0,15.186780,15.192469,-0.005688
490,Noah Anderson,Gold Coast,207.136345,205.099730,2.036617,1352,1607.0,19.0,12.889629,12.762895,0.126734
84,Caleb Serong,Fremantle,192.770327,197.288034,-4.517708,1436,1500.0,18.0,12.851355,13.152536,-0.301181
111,Christian Petracca,Melbourne,204.171447,211.672250,-7.500802,1377,1592.0,19.0,12.824840,13.295996,-0.471156
660,Zak Butters,Port Adelaide,187.002842,188.319077,-1.316234,1410,1495.0,19.0,12.508551,12.596594,-0.088042
148,Darcy Parish,Essendon,134.673054,146.496396,-11.823343,1170,1109.0,14.0,12.143648,13.209774,-1.066127
98,Chad Warner,Sydney,148.855603,156.961817,-8.106213,1083,1227.0,15.0,12.131671,12.792324,-0.660653
330,Jordan De Goey,Collingwood,153.439985,153.984431,-0.544446,913,1278.0,15.0,12.006259,12.048860,-0.042601
573,Shai Bolton,Richmond,197.169424,198.858498,-1.689074,1107,1645.0,19.0,11.985983,12.088662,-0.102679
578,Stephen Coniglio,Greater Western Sydney,170.722975,186.689445,-15.966470,1354,1481.0,18.0,11.527547,12.605634,-1.078087


In [57]:
player_ratings_2023.sort_values('exp_offensive_value', ascending = False).head(10)

,player,team,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
424,Marcus Bontempelli,Western Bulldogs,257.264062,257.360419,-0.096358,1418,1694.0,20.0,15.186780,15.192469,-0.005688
105,Charlie Curnow,Carlton,192.572502,211.673820,-19.101318,699,1774.0,19.0,10.855271,11.932008,-1.076737
111,Christian Petracca,Melbourne,204.171447,211.672250,-7.500802,1377,1592.0,19.0,12.824840,13.295996,-0.471156
490,Noah Anderson,Gold Coast,207.136345,205.099730,2.036617,1352,1607.0,19.0,12.889629,12.762895,0.126734
573,Shai Bolton,Richmond,197.169424,198.858498,-1.689074,1107,1645.0,19.0,11.985983,12.088662,-0.102679
84,Caleb Serong,Fremantle,192.770327,197.288034,-4.517708,1436,1500.0,18.0,12.851355,13.152536,-0.301181
660,Zak Butters,Port Adelaide,187.002842,188.319077,-1.316234,1410,1495.0,19.0,12.508551,12.596594,-0.088042
578,Stephen Coniglio,Greater Western Sydney,170.722975,186.689445,-15.966470,1354,1481.0,18.0,11.527547,12.605634,-1.078087
320,Joe Daniher,Brisbane Lions,166.102024,180.229992,-14.127968,736,1583.0,18.0,10.492863,11.385344,-0.892481
329,Jordan Dawson,Adelaide,167.619776,172.826525,-5.206748,1372,1635.0,19.0,10.251974,10.570430,-0.318456


In [58]:
player_ratings_2023.sort_values('offensive_rating', ascending = False).head(10)

,player,team,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
424,Marcus Bontempelli,Western Bulldogs,257.264062,257.360419,-0.096358,1418,1694.0,20.0,15.186780,15.192469,-0.005688
111,Christian Petracca,Melbourne,204.171447,211.672250,-7.500802,1377,1592.0,19.0,12.824840,13.295996,-0.471156
148,Darcy Parish,Essendon,134.673054,146.496396,-11.823343,1170,1109.0,14.0,12.143648,13.209774,-1.066127
84,Caleb Serong,Fremantle,192.770327,197.288034,-4.517708,1436,1500.0,18.0,12.851355,13.152536,-0.301181
98,Chad Warner,Sydney,148.855603,156.961817,-8.106213,1083,1227.0,15.0,12.131671,12.792324,-0.660653
490,Noah Anderson,Gold Coast,207.136345,205.099730,2.036617,1352,1607.0,19.0,12.889629,12.762895,0.126734
578,Stephen Coniglio,Greater Western Sydney,170.722975,186.689445,-15.966470,1354,1481.0,18.0,11.527547,12.605634,-1.078087
660,Zak Butters,Port Adelaide,187.002842,188.319077,-1.316234,1410,1495.0,19.0,12.508551,12.596594,-0.088042
573,Shai Bolton,Richmond,197.169424,198.858498,-1.689074,1107,1645.0,19.0,11.985983,12.088662,-0.102679
330,Jordan De Goey,Collingwood,153.439985,153.984431,-0.544446,913,1278.0,15.0,12.006259,12.048860,-0.042601


In [59]:
player_ratings_2023.sort_values('exp_defensive_value', ascending = False).head(10)

,player,team,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
226,Jack Buckley,Greater Western Sydney,139.226504,61.917756,77.308749,925,1681.0,18.0,8.282362,3.683388,4.598974
193,Harris Andrews,Brisbane Lions,151.203150,73.971234,77.231915,1031,1756.0,18.0,8.610658,4.212485,4.398173
546,Sam Collins,Gold Coast,121.168150,48.989126,72.179023,893,1855.0,19.0,6.531976,2.640923,3.891052
18,Aliir Aliir,Port Adelaide,112.556034,43.486038,69.069995,830,1855.0,19.0,6.067711,2.344261,3.723450
562,Sam Taylor,Greater Western Sydney,89.460772,24.227948,65.232823,639,1003.0,11.0,8.919319,2.415548,6.503771
487,Nick Murray,Adelaide,92.224409,33.139238,59.085171,520,1353.0,17.0,6.816290,2.449315,4.366975
603,Tom Barrass,West Coast,78.114412,23.220312,54.894100,572,1341.0,14.0,5.825087,1.731567,4.093520
101,Charlie Ballard,Gold Coast,105.543935,52.205107,53.338828,861,1815.0,19.0,5.815093,2.876314,2.938778
261,Jacob Weitering,Carlton,100.003982,48.099328,51.904655,994,1881.0,19.0,5.316533,2.557115,2.759418
147,Darcy Moore,Collingwood,122.809142,72.200899,50.608243,1003,1693.0,18.0,7.253936,4.264672,2.989264


In [60]:
player_ratings_2023.sort_values('defensive_rating', ascending = False).head(10)

,player,team,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
562,Sam Taylor,Greater Western Sydney,89.460772,24.227948,65.232823,639,1003.0,11.0,8.919319,2.415548,6.503771
226,Jack Buckley,Greater Western Sydney,139.226504,61.917756,77.308749,925,1681.0,18.0,8.282362,3.683388,4.598974
193,Harris Andrews,Brisbane Lions,151.203150,73.971234,77.231915,1031,1756.0,18.0,8.610658,4.212485,4.398173
487,Nick Murray,Adelaide,92.224409,33.139238,59.085171,520,1353.0,17.0,6.816290,2.449315,4.366975
603,Tom Barrass,West Coast,78.114412,23.220312,54.894100,572,1341.0,14.0,5.825087,1.731567,4.093520
546,Sam Collins,Gold Coast,121.168150,48.989126,72.179023,893,1855.0,19.0,6.531976,2.640923,3.891052
18,Aliir Aliir,Port Adelaide,112.556034,43.486038,69.069995,830,1855.0,19.0,6.067711,2.344261,3.723450
178,Esava Ratugolea,Geelong,60.940689,25.488045,35.452645,451,1139.0,13.0,5.350368,2.237756,3.112611
69,Brandon Zerk-Thatcher,Essendon,81.973701,33.099390,48.874310,806,1612.0,18.0,5.085217,2.053312,3.031905
147,Darcy Moore,Collingwood,122.809142,72.200899,50.608243,1003,1693.0,18.0,7.253936,4.264672,2.989264


In [61]:
action_vaep = get_vaep_action_summary(exp_vaep_2021)
action_vaep.sort_values("exp_vaep_value_per_action")

,action_type,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions,action_%,exp_vaep_value_per_action,off_value_per_action,def_value_per_action,vaep_%,offensive_%,defensive_%
2,Error,-707.746615,-402.267515,-305.479102,972,0.002816,-0.728134,-0.413855,-0.314279,-0.019739,-0.011474,-0.383790
12,Tackle,-9.762774,3.460380,-13.223153,129,0.000374,-0.075680,0.026825,-0.102505,-0.000272,0.000099,-0.016613
5,Handball,-2509.008210,-717.289126,-1791.719077,60799,0.176132,-0.041267,-0.011798,-0.029470,-0.069974,-0.020459,-2.251037
7,Kick,3145.090539,5866.859728,-2721.769180,80643,0.233619,0.039000,0.072751,-0.033751,0.087714,0.167337,-3.419510
0,Carry,3752.219008,4871.426798,-1119.207785,54634,0.158272,0.068679,0.089165,-0.020486,0.104647,0.138945,-1.406123
11,Spoil,1384.601283,115.968894,1268.632386,13590,0.039370,0.101884,0.008533,0.093350,0.038616,0.003308,1.593854
4,Gather,1750.041960,1661.448669,88.593290,14029,0.040641,0.124745,0.118430,0.006315,0.048807,0.047389,0.111305
6,Hard Ball Get,2914.309275,1813.254638,1101.054633,18947,0.054889,0.153814,0.095701,0.058112,0.081278,0.051718,1.383316
13,Uncontested Mark,6649.244029,6348.789125,300.454907,38669,0.112022,0.171953,0.164183,0.007770,0.185443,0.181083,0.377478
10,Shot,1994.796648,2881.644250,-886.847601,9834,0.028489,0.202847,0.293029,-0.090182,0.055633,0.082192,-1.114196


In [62]:
action_vaep = get_vaep_action_summary(exp_vaep_2022)
action_vaep.sort_values("exp_vaep_value_per_action")

,action_type,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions,action_%,exp_vaep_value_per_action,off_value_per_action,def_value_per_action,vaep_%,offensive_%,defensive_%
2,Error,-889.930222,-502.843876,-387.086346,1198,0.003503,-0.742847,-0.419736,-0.323110,-0.024527,-0.014082,-0.674525
12,Tackle,-31.940400,-12.500703,-19.439697,84,0.000246,-0.380243,-0.148818,-0.231425,-0.000880,-0.000350,-0.033875
5,Handball,-2522.717950,-658.979440,-1863.738510,59270,0.173288,-0.042563,-0.011118,-0.031445,-0.069529,-0.018454,-3.247693
7,Kick,3001.215980,5772.888931,-2771.672947,79965,0.233794,0.037532,0.072193,-0.034661,0.082717,0.161664,-4.829831
0,Carry,3443.309995,4582.521005,-1139.211011,53272,0.155752,0.064636,0.086021,-0.021385,0.094901,0.128329,-1.985154
11,Spoil,1450.268258,98.474308,1351.793949,13964,0.040827,0.103858,0.007052,0.096806,0.039971,0.002758,2.355594
4,Gather,1898.299109,1739.095403,159.203705,13390,0.039148,0.141770,0.129880,0.011890,0.052319,0.048702,0.277423
6,Hard Ball Get,2821.583280,1825.893134,995.690148,17178,0.050223,0.164256,0.106293,0.057963,0.077766,0.051132,1.735059
13,Uncontested Mark,6895.145322,6560.604844,334.540494,38101,0.111396,0.180970,0.172190,0.008780,0.190037,0.183723,0.582960
10,Shot,2037.418284,2965.843124,-928.424838,10069,0.029439,0.202346,0.294552,-0.092206,0.056153,0.083055,-1.617844


In [63]:
action_vaep = get_vaep_action_summary(exp_vaep_2023)
action_vaep.sort_values("exp_vaep_value_per_action")

,action_type,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions,action_%,exp_vaep_value_per_action,off_value_per_action,def_value_per_action,vaep_%,offensive_%,defensive_%
2,Error,-773.315703,-431.633781,-341.681922,1068,0.003096,-0.724078,-0.404151,-0.319927,-0.021118,-0.011925,-0.805422
12,Tackle,-13.310125,-0.190272,-13.119854,101,0.000293,-0.131783,-0.001884,-0.129900,-0.000363,-0.000005,-0.030926
5,Handball,-2767.528211,-781.601257,-1985.926960,61695,0.178862,-0.044858,-0.012669,-0.032189,-0.075576,-0.021594,-4.681279
7,Kick,2969.340036,5936.158043,-2966.818005,79022,0.229095,0.037576,0.075120,-0.037544,0.081087,0.164006,-6.993461
0,Carry,4017.194774,5070.247843,-1053.053071,55393,0.160592,0.072522,0.091532,-0.019011,0.109702,0.140082,-2.482284
11,Spoil,1378.763983,70.928895,1307.835089,13665,0.039617,0.100897,0.005191,0.095707,0.037652,0.001960,3.082863
4,Gather,1785.926272,1650.240621,135.685649,13573,0.039350,0.131579,0.121583,0.009997,0.048770,0.045593,0.319842
6,Hard Ball Get,2629.420425,1676.718851,952.701571,16277,0.047189,0.161542,0.103012,0.058531,0.071805,0.046325,2.245733
13,Uncontested Mark,7282.497306,6910.419068,372.078234,38100,0.110457,0.191142,0.181376,0.009766,0.198872,0.190923,0.877072
10,Shot,1996.596619,2936.539232,-939.942608,10242,0.029693,0.194942,0.286715,-0.091773,0.054523,0.081131,-2.215657


In [64]:
player = "Nick Daicos"
player_data = get_player(exp_vaep_2022, player)
player_action_vaep = get_vaep_action_summary(player_data)
player_action_vaep.sort_values("exp_vaep_value")

,action_type,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions,action_%,exp_vaep_value_per_action,off_value_per_action,def_value_per_action,vaep_%,offensive_%,defensive_%
4,Handball,-20.410731,-3.802585,-16.608145,283,0.198457,-0.072123,-0.013437,-0.058686,-0.259258,-0.036168,0.628866
1,Error,-4.748038,-2.053168,-2.694870,6,0.004208,-0.791340,-0.342195,-0.449145,-0.060310,-0.019528,0.102041
10,Spoil,-0.794625,-0.418953,-0.375673,5,0.003506,-0.158925,-0.083791,-0.075135,-0.010093,-0.003985,0.014225
7,Knock On,1.072689,0.890468,0.182222,4,0.002805,0.268172,0.222617,0.045555,0.013625,0.008470,-0.006900
3,Gather,3.836689,2.488436,1.348252,44,0.030856,0.087197,0.056555,0.030642,0.048734,0.023668,-0.051051
5,Hard Ball Get,6.360983,3.765575,2.595408,47,0.032959,0.135340,0.080119,0.055221,0.080797,0.035816,-0.098275
9,Shot,6.991208,8.187622,-1.196414,17,0.011921,0.411248,0.481625,-0.070377,0.088803,0.077876,0.045302
2,Free For,11.650791,7.898440,3.752351,28,0.019635,0.416100,0.282087,0.134013,0.147989,0.075125,-0.142083
6,Kick,13.119107,27.112305,-13.993197,344,0.241234,0.038137,0.078815,-0.040678,0.166639,0.257875,0.529851
11,Uncontested Mark,15.953058,15.224683,0.728375,124,0.086957,0.128654,0.122780,0.005874,0.202636,0.144808,-0.027580


In [65]:
player = "Nick Daicos"
player_data = get_player(exp_vaep_2023, player)
player_action_vaep = get_vaep_action_summary(player_data)
player_action_vaep.sort_values("exp_vaep_value")

,action_type,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions,action_%,exp_vaep_value_per_action,off_value_per_action,def_value_per_action,vaep_%,offensive_%,defensive_%
4,Handball,-19.482414,-6.954399,-12.528015,300,0.226929,-0.064941,-0.023181,-0.041760,-0.147500,-0.050806,2.610982
1,Error,-1.425278,-0.968864,-0.456414,3,0.002269,-0.475093,-0.322955,-0.152138,-0.010791,-0.007078,0.095122
7,Knock On,-0.115535,-0.157725,0.042190,2,0.001513,-0.057768,-0.078863,0.021095,-0.000875,-0.001152,-0.008793
10,Spoil,0.039300,0.408196,-0.368896,3,0.002269,0.013100,0.136065,-0.122965,0.000298,0.002982,0.076882
5,Hard Ball Get,7.691305,7.390658,0.300648,30,0.022693,0.256377,0.246355,0.010022,0.058230,0.053993,-0.062658
9,Shot,8.088309,11.657753,-3.569445,39,0.029501,0.207393,0.298917,-0.091524,0.061236,0.085166,0.743913
2,Free For,13.292969,12.182624,1.110345,24,0.018154,0.553874,0.507609,0.046264,0.100640,0.089001,-0.231409
11,Uncontested Mark,15.392200,15.490767,-0.098567,70,0.052950,0.219889,0.221297,-0.001408,0.116533,0.113169,0.020543
6,Kick,17.319048,21.339614,-4.020566,286,0.216339,0.060556,0.074614,-0.014058,0.131121,0.155898,0.837932
3,Gather,18.817895,14.643040,4.174854,59,0.044629,0.318947,0.248187,0.070760,0.142469,0.106975,-0.870088


In [67]:
player_ratings_2023[player_ratings_2023['player'] == 'Nick Daicos']

,player,team,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
482,Nick Daicos,Collingwood,132.084004,136.882204,-4.7982,1322,1541.0,18.0,8.571318,8.882687,-0.311369


In [68]:
player = "Christian Petracca"
player_data = get_player(exp_vaep_2022, player)
player_action_vaep = get_vaep_action_summary(player_data)
player_action_vaep.sort_values("exp_vaep_value")

,action_type,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions,action_%,exp_vaep_value_per_action,off_value_per_action,def_value_per_action,vaep_%,offensive_%,defensive_%
2,Error,-5.970762,-5.716873,-0.253890,6,0.004237,-0.995127,-0.952812,-0.042315,-0.027777,-0.027704,-0.029526
5,Handball,-5.511843,0.037149,-5.548992,316,0.223164,-0.017443,0.000118,-0.017560,-0.025642,0.000180,-0.645311
11,Spoil,0.555117,0.175283,0.379834,5,0.003531,0.111023,0.035057,0.075967,0.002583,0.000849,0.044172
8,Knock On,1.509850,0.600803,0.909047,6,0.004237,0.251642,0.100134,0.151508,0.007024,0.002912,0.105716
1,Contested Mark,1.803306,1.821298,-0.017992,4,0.002825,0.450827,0.455325,-0.004498,0.008389,0.008826,-0.002092
3,Free For,11.291936,10.156947,1.134989,16,0.011299,0.705746,0.634809,0.070937,0.052532,0.049221,0.131992
7,Kick,12.324515,17.769987,-5.445472,288,0.203390,0.042793,0.061701,-0.018908,0.057336,0.086114,-0.633273
12,Uncontested Mark,21.442891,20.973145,0.469745,103,0.072740,0.208183,0.203623,0.004561,0.099756,0.101637,0.054628
6,Hard Ball Get,24.892408,19.750145,5.142263,102,0.072034,0.244043,0.193629,0.050414,0.115804,0.095710,0.598011
10,Shot,30.166107,33.402543,-3.236437,69,0.048729,0.437190,0.484095,-0.046905,0.140339,0.161871,-0.376376


In [69]:
player = "Christian Petracca"
player_data = get_player(exp_vaep_2023, player)
player_action_vaep = get_vaep_action_summary(player_data)
player_action_vaep.sort_values("exp_vaep_value")

,action_type,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions,action_%,exp_vaep_value_per_action,off_value_per_action,def_value_per_action,vaep_%,offensive_%,defensive_%
5,Handball,-13.297134,-8.056845,-5.240289,321,0.233115,-0.041424,-0.025099,-0.016325,-0.065127,-0.038063,0.698630
2,Error,-7.734355,-3.497316,-4.237039,12,0.008715,-0.644530,-0.291443,-0.353087,-0.037882,-0.016522,0.564878
11,Spoil,0.561539,0.719688,-0.158149,6,0.004357,0.093590,0.119948,-0.026358,0.002750,0.003400,0.021084
8,Knock On,1.197438,1.147679,0.049759,2,0.001452,0.598719,0.573840,0.024879,0.005865,0.005422,-0.006634
1,Contested Mark,8.991330,9.198860,-0.207529,6,0.004357,1.498555,1.533143,-0.034588,0.044038,0.043458,0.027668
6,Hard Ball Get,10.309987,4.859856,5.450130,106,0.076979,0.097264,0.045848,0.051416,0.050497,0.022959,-0.726606
3,Free For,14.314772,12.487120,1.827653,20,0.014524,0.715739,0.624356,0.091383,0.070112,0.058993,-0.243661
7,Kick,23.321299,31.836270,-8.514971,242,0.175744,0.096369,0.131555,-0.035186,0.114224,0.150404,1.135208
10,Shot,26.154957,31.453648,-5.298691,79,0.057371,0.331075,0.398147,-0.067072,0.128103,0.148596,0.706417
0,Carry,26.792960,27.381389,-0.588429,194,0.140886,0.138108,0.141141,-0.003033,0.131228,0.129357,0.078449


In [70]:
player_ratings_2023[player_ratings_2023['player'] ==  "Christian Petracca"]

,player,team,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions,minutes_percent,games,vaep_rating,offensive_rating,defensive_rating
111,Christian Petracca,Melbourne,204.171447,211.67225,-7.500802,1377,1592.0,19.0,12.82484,13.295996,-0.471156


In [ ]:
team_data = get_team(exp_vaep_2022, "Geelong")
team_action_vaep = get_vaep_action_summary(team_data)
team_action_vaep.sort_values("exp_vaep_value")

In [ ]:
team_data = get_team(exp_vaep_2023, "Geelong")
team_action_vaep = get_vaep_action_summary(team_data)
team_action_vaep.sort_values("exp_vaep_value")

In [ ]:
team_data = get_team(exp_vaep_2023, "Collingwood")
team_action_vaep = get_vaep_action_summary(team_data)
team_action_vaep.sort_values("exp_vaep_value")

In [ ]:
team_data = get_team(exp_vaep_2023, "St Kilda")
team_action_vaep = get_vaep_action_summary(team_data)
team_action_vaep.sort_values("exp_vaep_value")

Match Expectd Scores v Expected VAEP

In [47]:
match_id = "2022F4_Geelong_Sydney"
match_vaep_ranking = get_match_player_level_vaep(exp_vaep_data, match_id)
match_vaep_ranking.sort_values('exp_vaep_value', ascending = False).head()

,player,team,exp_vaep_value,exp_offensive_value,exp_defensive_value,num_actions,vaep_ranking,offensive_ranking,defensive_ranking,normalised_exp_vaep_value,normalised_exp_offensive_value,normalised_exp_defensive_value,z_exp_vaep_value,z_exp_offensive_value,z_exp_defensive_value
30,Patrick Dangerfield,Geelong,15.696886,15.664802,0.032084,58,1.0,1.0,19.0,1.000000,1.000000,0.440498,3.328965,3.582580,0.183025
24,Mark Blicavs,Geelong,15.555391,13.423626,2.131765,49,2.0,2.0,5.0,0.992543,0.865812,0.694816,3.292052,2.938175,1.459669
5,Chad Warner,Sydney,9.248642,10.645956,-1.397314,69,3.0,3.0,36.0,0.660174,0.699503,0.267366,1.646793,2.139511,-0.686075
13,Isaac Smith,Geelong,7.629659,8.877090,-1.247431,69,4.0,4.0,34.0,0.574852,0.593594,0.285520,1.224444,1.630909,-0.594944
45,Zach Tuohy,Geelong,6.944715,2.293307,4.651408,49,5.0,23.0,1.0,0.538755,0.199398,1.000000,1.045761,-0.262125,2.991658


In [49]:
match_chain = get_match(exp_vaep_data, match_id)
match_chain.head()

,match_id,chain_number,order,quarter,quarter_seconds,overall_seconds,team,player,start_x,start_y,end_x,end_y,action_type,outcome_type,xScore,exp_scores,exp_concedes,exp_offensive_value,exp_defensive_value,exp_vaep_value,Season
685607,2022F4_Geelong_Sydney,1,2.0,1,5.0,5.0,Geelong,Patrick Dangerfield,1.0,-2.0,1.0,-2.0,Gather,effective,NaN,0.429417,0.082917,0.000000,-0.000000,0.000000,2022
685608,2022F4_Geelong_Sydney,1,3.0,1,5.0,5.0,Geelong,Patrick Dangerfield,1.0,-2.0,17.0,-3.0,Kick,ineffective,NaN,1.000678,0.071120,0.571261,0.011797,0.583059,2022
685609,2022F4_Geelong_Sydney,1,4.0,1,8.0,8.0,Geelong,Rhys Stanley,17.0,-3.0,18.0,-2.0,Loose Ball Get,effective,NaN,1.217941,0.060284,0.217262,0.010836,0.228098,2022
685610,2022F4_Geelong_Sydney,1,5.0,1,9.0,9.0,Geelong,Rhys Stanley,18.0,-2.0,-21.0,-3.0,Handball,clanger,NaN,0.552853,0.141398,-0.665088,-0.081114,-0.746202,2022
685611,2022F4_Geelong_Sydney,1,6.0,1,13.0,13.0,Sydney,Luke Parker,-21.0,-3.0,-25.0,-6.0,Hard Ball Get,effective,NaN,0.106199,0.286399,-0.035200,0.266454,0.231255,2022


In [52]:
match_chain.groupby('team').sum()[['xScore','exp_scores', 'exp_concedes', 'exp_vaep_value', 'exp_offensive_value', 'exp_defensive_value']]

,xScore,exp_scores,exp_concedes,exp_vaep_value,exp_offensive_value,exp_defensive_value
team,,,,,,
Geelong,105.861173,832.788611,100.947935,96.362866,94.129560,2.233307
Sydney,42.689429,417.460013,160.147269,38.693912,53.298192,-14.604281


In [51]:
match_shots = match_chain[match_chain['xScore'] > 0]
match_shots.groupby('team').sum()[['xScore','exp_scores', 'exp_concedes', 'exp_vaep_value', 'exp_offensive_value', 'exp_defensive_value']]

,xScore,exp_scores,exp_concedes,exp_vaep_value,exp_offensive_value,exp_defensive_value
team,,,,,,
Geelong,105.861173,105.628644,6.518908,7.951791,11.845054,-3.893262
Sydney,42.689429,41.479330,3.440127,4.196166,6.802696,-2.606530


In [ ]:
shots = exp_vaep_data[exp_vaep_data['xScore'] > 0]
shots.sum()[['xScore','exp_scores', 'exp_concedes', 'exp_vaep_value', 'exp_offensive_value', 'exp_defensive_value']]

In [ ]:
exp_vaep_data.sum()[['xScore','exp_scores', 'exp_concedes', 'exp_vaep_value', 'exp_offensive_value', 'exp_defensive_value']]

In [54]:
exp_vaep_data.groupby('match_id').sum()[['xScore','exp_scores', 'exp_concedes', 'exp_vaep_value', 'exp_offensive_value', 'exp_defensive_value']]

,xScore,exp_scores,exp_concedes,exp_vaep_value,exp_offensive_value,exp_defensive_value
match_id,,,,,,
202101_BrisbaneLions_Sydney,162.196572,1274.995573,252.989036,174.166985,173.001272,1.165714
202101_Collingwood_WesternBulldogs,150.357535,1409.072050,273.537137,164.917819,159.650109,5.267709
202101_Essendon_Hawthorn,183.807470,1461.791648,304.084892,179.750145,183.144667,-3.394521
202101_GreaterWesternSydney_StKilda,172.332941,1288.079378,306.514631,198.108735,188.956258,9.152477
202101_Melbourne_Fremantle,166.657732,1320.673314,278.068566,153.401035,159.781924,-6.380888
202101_NorthMelbourne_PortAdelaide,199.081580,1375.849002,280.060125,206.945651,205.812376,1.133275
202101_Richmond_Carlton,206.995237,1437.942825,271.171508,201.156198,207.918084,-6.761886
202101_WestCoast_GoldCoast,164.258414,1206.146890,249.739386,181.441277,175.620522,5.820756
202102_Carlton_Collingwood,166.299330,1375.268083,268.868758,206.482581,189.721235,16.761347
